In [14]:
import os
from IPython.display import IFrame
import pandas as pd
import sys
import igv
import numpy as np
import pyBigWig
import sys
sys.path.insert(0, '../../')
from JKBio import TerraFunction as terra
from ccle_processing.src.CCLE_postp_function import *
from JKBio import Helper as h
from JKBio.epigenetics import ChIP_helper as chip
import dalmatian as dm
import itertools
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from bokeh.plotting import *
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

In [95]:
chip

<module 'JKBio.epigenetics.ChIP_helper' from '/home/jeremie/JKBio/epigenetics/ChIP_helper.py'>

# ChIP

In [ ]:
!gsutil ls gs://amlproject/Chip_AML/Chip/

In [ ]:
a = !gsutil ls gs://amlproject/Chip_AML/Chip/IRF2BP2/

In [ ]:
rename1 = {
 'gs://transfer-amlproject/20191211_10_MP7613_S8_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp910_1.fastq.gz",
 'gs://transfer-amlproject/20191211_10_MP7613_S8_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp910_2.fastq.gz",
 'gs://transfer-amlproject/20191211_11_MP7613_S9_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp911_1.fastq.gz",
 'gs://transfer-amlproject/20191211_11_MP7613_S9_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp911_2.fastq.gz",
 'gs://transfer-amlproject/20191211_12_MP7613_S10_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp912_1.fastq.gz",
 'gs://transfer-amlproject/20191211_12_MP7613_S10_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp912_2.fastq.gz",
 'gs://transfer-amlproject/20191211_13_MP7613_S11_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp913_1.fastq.gz",
 'gs://transfer-amlproject/20191211_13_MP7613_S11_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp913_2.fastq.gz",
 'gs://transfer-amlproject/20191211_14_MP7613_S12_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp914_1.fastq.gz",
 'gs://transfer-amlproject/20191211_14_MP7613_S12_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp914_2.fastq.gz",
 'gs://transfer-amlproject/20191211_15_MP7613_S13_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp915_1.fastq.gz",
 'gs://transfer-amlproject/20191211_15_MP7613_S13_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp915_2.fastq.gz",
 'gs://transfer-amlproject/20191211_16_MP7613_S14_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp916_1.fastq.gz",
 'gs://transfer-amlproject/20191211_16_MP7613_S14_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp916_2.fastq.gz",
 'gs://transfer-amlproject/20191211_1_MP7613_S1_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp901_1.fastq.gz",
 'gs://transfer-amlproject/20191211_1_MP7613_S1_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp901_2.fastq.gz",
 'gs://transfer-amlproject/20191211_2_MP7613_S2_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp902_1.fastq.gz",
 'gs://transfer-amlproject/20191211_2_MP7613_S2_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp902_2.fastq.gz",
 'gs://transfer-amlproject/20191211_3_MP7613_S3_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp903_1.fastq.gz",
 'gs://transfer-amlproject/20191211_3_MP7613_S3_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp903_2.fastq.gz",
 'gs://transfer-amlproject/20191211_4_MP7613_S4_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp904_1.fastq.gz",
 'gs://transfer-amlproject/20191211_4_MP7613_S4_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp904_2.fastq.gz",
 'gs://transfer-amlproject/20191211_5R_MP7613_S15_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp905_1.fastq.gz",
 'gs://transfer-amlproject/20191211_5R_MP7613_S15_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp905_2.fastq.gz",
 'gs://transfer-amlproject/20191211_6R_MP7613_S16_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp906_1.fastq.gz",
 'gs://transfer-amlproject/20191211_6R_MP7613_S16_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp906_2.fastq.gz",
 'gs://transfer-amlproject/20191211_7_MP7613_S5_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp907_1.fastq.gz",
 'gs://transfer-amlproject/20191211_7_MP7613_S5_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp907_2.fastq.gz",
 'gs://transfer-amlproject/20191211_8_MP7613_S6_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp908_1.fastq.gz",
 'gs://transfer-amlproject/20191211_8_MP7613_S6_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp908_2.fastq.gz",
 'gs://transfer-amlproject/20191211_9_MP7613_S7_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp909_1.fastq.gz",
 'gs://transfer-amlproject/20191211_9_MP7613_S7_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp909_2.fastq.gz"}

In [ ]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [ ]:
! mkdir bams && gsutil -m cp "gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/*" bams

In [ ]:
inputfastq="gs://amlproject/Chip_AML/Chip/mp99.fastq.gz"
! gsutil cp $inputfastq bams
a = ! ls bams

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
gsheet

In [ ]:
df = {
"fastq1": [],
"fastq2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-1],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('_')[0]]
    df['group'].append((row.id.values[0] + '_'+ row.name_replicate.values[0] +"_"+row.protein.values[0]))
    df['replicate'].append(1)
    df['fastq1'].append(val[0])
    #df['fastq2'].append(val[1])
    df['antibody'].append(row.protein.values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq1'].append(a[-1])
#df['fastq2'].append(val[1])
df['antibody'].append("INPUT")
df['control'].append("INPUT")
df = pd.DataFrame(df)

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv')

In [ ]:
#process chips
! sudo ./nextflow run nf-core/chipseq --single_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --narrow_peak --input design.csv --genome GRCh38 --skip_preseq --max_cpus 24 -profile docker -w work

In [ ]:
#!gsutil cp results gs://amlproject/Chip_AML/Chip/IRF2BP2/ && mv results results1 && sudo rm -r work

In [ ]:
# install bwa
! mkdir ref
! curl ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/215/GCF_000001215.4_Release_6_plus_ISO1_MT/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.gz -o ref/reference_droso.fna.gz
! gunzip ref/reference_droso.fna.gz
! bwa index -a bwtsw ref/reference_droso.fna

In [ ]:
# get scaling values
mappedreads, umappedreads_norm, mapped = chip.getSpikeInControlScales(refgenome="ref/reference_droso.fna",FastQfolder='bams',pairedEnd=True, cores=8)
mappedreads, umappedreads_norm, mapped

computing scales from the excel sheet

In [ ]:
scales = [1.0, 0.3756814126],
[1.0, 0.2342366196],
[0.5546432407, 1.0],
[0.1601166841, 1.0],
[0.870315477, 1.0],
[1.0, 0.8742449687],
[0.7172728398, 1.0],
[1.0, 0.7713982525]

In [ ]:
# diffPeak & 
bams = !ls results1/bwa/mergedLibrary/*.bam
for i in range(int(len(bams[1:])/2)-1):
    name1 = bams[1+val]
    name2 = bams[9+val]
    chip.fullDiffPeak(name1,name2, bams[0], scaling = scales[val])

In [ ]:
scales = [1.0,
1.0,
0.5546432407,
0.1601166841,
0.870315477,
1.0,
0.7172728398,
1.0,
0.3756814126,
0.2342366196,
1.0,
1.0,
1.0,
0.8742449687,
1.0,
0.7713982525]

In [ ]:
chip.bigWigFrom(bams[1:],genome='GRCh38',scaling=scales)

In [ ]:
! gsutil -m cp -r "diffPeaks" gs://amlproject/Chip/IRF2BP2/ &&\
    gsutil -m cp -r "diffData" gs://amlproject/Chip/IRF2BP2/ &&\
        gsutil -m cp -r "bigwigs" gs://amlproject/Chip/IRF2BP2/

## v2

In [ ]:
#! rm -r bams && mkdir bams && gsutil -m cp "gs://transfer-amlproject/20191219_MP7659*" bams/

In [ ]:
rename = {
"20191219_MP7659_1_S1_R1_001.fastq.gz":"mp795_R1.fastq.gz",
"20191219_MP7659_1_S1_R2_001.fastq.gz":"mp795_R2.fastq.gz",
"20191219_MP7659_2_S2_R1_001.fastq.gz":"mp796_R1.fastq.gz",
"20191219_MP7659_2_S2_R2_001.fastq.gz":"mp796_R2.fastq.gz",
"20191219_MP7659_3_S3_R1_001.fastq.gz":"mp797_R1.fastq.gz",
"20191219_MP7659_3_S3_R2_001.fastq.gz":"mp797_R2.fastq.gz",
"20191219_MP7659_4_S4_R1_001.fastq.gz":"mp798_R1.fastq.gz",
"20191219_MP7659_4_S4_R2_001.fastq.gz":"mp798_R2.fastq.gz",
"20191219_MP7659_5_S5_R1_001.fastq.gz":"mp799_R1.fastq.gz",
"20191219_MP7659_5_S5_R2_001.fastq.gz":"mp799_R2.fastq.gz",
"20191219_MP7659_6_S6_R1_001.fastq.gz":"mp800_R1.fastq.gz",
"20191219_MP7659_6_S6_R2_001.fastq.gz":"mp800_R2.fastq.gz",
"20191219_MP7659_7_S7_R1_001.fastq.gz":"mp801_R1.fastq.gz",
"20191219_MP7659_7_S7_R2_001.fastq.gz":"mp801_R2.fastq.gz",
"20191219_MP7659_8_S8_R1_001.fastq.gz":"mp802_R1.fastq.gz",
"20191219_MP7659_8_S8_R2_001.fastq.gz":"mp802_R2.fastq.gz",
"20191219_MP7659_9_S9_R1_001.fastq.gz":"mp803_R1.fastq.gz",
"20191219_MP7659_9_S9_R2_001.fastq.gz":"mp803_R2.fastq.gz",
"20191219_MP7659_10_S10_R1_001.fastq.gz":"mp804_R1.fastq.gz",
"20191219_MP7659_10_S10_R2_001.fastq.gz":"mp804_R2.fastq.gz",
"20191219_MP7659_11_S11_R1_001.fastq.gz":"mp805_R1.fastq.gz",
"20191219_MP7659_11_S11_R2_001.fastq.gz":"mp805_R2.fastq.gz",
"20191219_MP7659_12_S12_R1_001.fastq.gz":"mp806_R1.fastq.gz",
"20191219_MP7659_12_S12_R2_001.fastq.gz":"mp806_R2.fastq.gz",
"20191219_MP7659_13_S13_R1_001.fastq.gz":"mp807_R1.fastq.gz",
"20191219_MP7659_13_S13_R2_001.fastq.gz":"mp807_R2.fastq.gz",
"20191219_MP7659_14_S14_R1_001.fastq.gz":"mp808_R1.fastq.gz",
"20191219_MP7659_14_S14_R2_001.fastq.gz":"mp808_R2.fastq.gz",
"20191219_MP7659_15_S15_R1_001.fastq.gz":"mp809_R1.fastq.gz",
"20191219_MP7659_15_S15_R2_001.fastq.gz":"mp809_R2.fastq.gz",
"20191219_MP7659_16_S16_R1_001.fastq.gz":"mp810_R1.fastq.gz",
"20191219_MP7659_16_S16_R2_001.fastq.gz":"mp810_R2.fastq.gz"
}

In [ ]:
for k,v in rename.items():
    ! mv $k $v
! gsutil -m cp 'bams/*' gs://amlproject/Chip_AML/IRF2BP2/results_rep2/    

In [ ]:
! gsutil ls gs://amlproject/Chip_AML/IRF2BP2/

In [ ]:
inputfastq="gs://amlproject/Chip_AML/Chip/mp99.fastq.gz"
! gsutil cp $inputfastq bams
a = ! ls bams

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
df = {
"fastq1": [],
"fastq2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-1],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('_')[0]]
    df['group'].append((row.id.values[0] + '_'+ row.name_replicate.values[0] +"_"+row.protein.values[0]))
    df['replicate'].append(1)
    df['fastq1'].append(val[0])
    #df['fastq2'].append(val[1])
    df['antibody'].append(row.protein.values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq1'].append(a[-1])
#df['fastq2'].append(val[1])
df['antibody'].append("INPUT")
df['control'].append("INPUT")
df = pd.DataFrame(df)

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv')

In [ ]:
#process chips
! sudo ./nextflow run nf-core/chipseq --single_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --narrow_peak --input design.csv --genome GRCh38 --skip_preseq --max_cpus 24 -profile docker -w work

In [ ]:
#!gsutil cp results gs://amlproject/Chip_AML/IRF2BP2/ && sudo rm -r work && mv results results2

In [ ]:
# get scaling values
mappedreads, umappedreads_norm, mapped = chip.getSpikeInControlScales(refgenome="ref/reference_droso.fna",FastQfolder='bams',pairedEnd=True, cores=8)
mappedreads, umappedreads_norm, mapped

computing scales from the excel sheet

In [ ]:
scales = [1.0,0.7976494266],
[0.0060104845,1.0],
[0.6415816327,1.0],
[0.6370573492,1.0],
[1.0,0.8327053276],
[1.0,0.2711054926],
[0.8111195721,1.0],
[1.0,0.8338050685],

In [ ]:
# diffPeak & 
bams = ! ls results2/bwa/mergedLibrary/*.bam
for i in range(int(len(bams[1:])/2)-1):
    name1 = bams[1+val]
    name2 = bams[9+val]
    chip.fullDiffPeak(name1,name2, bams[0], scaling = scales[val])

In [ ]:
scales = [1.0,
0.006010484524,
0.6415816327,
0.6370573492,
1.0,
1.0,
0.8111195721,
1.0,
0.7976494266,
1.0,
1.0,
1.0,
0.8327053276,
0.2711054926,
1.0,
0.8338050685]

In [ ]:
chip.bigWigFrom(bams[1:],genome='GRCh38',scaling=scales)

In [ ]:
! gsutil -m cp -r "diffPeaks" gs://amlproject/Chip_AML/IRF2BP2/ &&\
    gsutil -m cp -r "diffData" gs://amlproject/Chip_AML/IRF2BP2/ &&\
        gsutil -m cp "bigwigs/mp*" gs://amlproject/Chip_AML/IRF2BP2/bigwigs

### histones V1&2

In [ ]:
bamfolder="histone_bams/"

In [ ]:
! mkdir histone_bams && gsutil cp "gs://transfer-amlproject/*.fastq.gz" $bamfolder

In [3]:
! gsutil -m cp gs://transfer-amlproject/*MP7693* gs://transfer-amlproject/IRF2BP2_hist/ && gsutil -m rm gs://transfer-amlproject/*MP7693*

Copying gs://transfer-amlproject/20200103_10_MP7693_S15_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200103_10_MP7693_S15_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200103_11_MP7693_S16_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200103_11_MP7693_S16_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200103_12_MP7693_S17_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200103_12_MP7693_S17_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200103_13_MP7693_S18_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200103_14_MP7693_S19_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200103_14_MP7693_S19_R2_001.fastq.gz [Content

In [ ]:
rename = {
    "20200103_1_MP7693_S6_R1_001.fastq.gz":"mp811_R1.fastq.gz",
    "20200103_1_MP7693_S6_R2_001.fastq.gz":"mp811_R2.fastq.gz",
    "20200103_2_MP7693_S7_R1_001.fastq.gz":"mp812_R1.fastq.gz",
    "20200103_2_MP7693_S7_R2_001.fastq.gz":"mp812_R2.fastq.gz",
    "20200103_3_MP7693_S8_R1_001.fastq.gz":"mp813_R1.fastq.gz",
    "20200103_3_MP7693_S8_R2_001.fastq.gz":"mp813_R2.fastq.gz",
    "20200103_4_MP7693_S9_R1_001.fastq.gz":"mp814_R1.fastq.gz",
    "20200103_4_MP7693_S9_R2_001.fastq.gz":"mp814_R2.fastq.gz",
    "20200103_5_MP7693_S10_R1_001.fastq.gz":"mp815_R1.fastq.gz",
    "20200103_5_MP7693_S10_R2_001.fastq.gz":"mp815_R2.fastq.gz",
    "20200103_6_MP7693_S11_R1_001.fastq.gz":"mp816_R1.fastq.gz",
    "20200103_6_MP7693_S11_R2_001.fastq.gz":"mp816_R2.fastq.gz",
    "20200103_7_MP7693_S12_R1_001.fastq.gz":"mp817_R1.fastq.gz",
    "20200103_7_MP7693_S12_R2_001.fastq.gz":"mp817_R2.fastq.gz",
    "20200103_8_MP7693_S13_R1_001.fastq.gz":"mp818_R1.fastq.gz",
    "20200103_8_MP7693_S13_R2_001.fastq.gz":"mp818_R2.fastq.gz",
    "20200103_9_MP7693_S14_R1_001.fastq.gz":"mp819_R1.fastq.gz",
    "20200103_9_MP7693_S14_R2_001.fastq.gz":"mp819_R2.fastq.gz",
    "20200103_10_MP7693_S15_R1_001.fastq.gz":"mp820_R1.fastq.gz",
    "20200103_10_MP7693_S15_R2_001.fastq.gz":"mp820_R2.fastq.gz",
    "20200103_11_MP7693_S16_R1_001.fastq.gz":"mp821_R1.fastq.gz",
    "20200103_11_MP7693_S16_R2_001.fastq.gz":"mp821_R2.fastq.gz",
    "20200103_12_MP7693_S17_R1_001.fastq.gz":"mp822_R1.fastq.gz",
    "20200103_12_MP7693_S17_R2_001.fastq.gz":"mp822_R2.fastq.gz",
    "20200103_13_MP7693_S18_R1_001.fastq.gz":"mp823_R1.fastq.gz",
    "20200103_13_MP7693_S18_R2_001.fastq.gz":"mp823_R2.fastq.gz",
    "20200103_14_MP7693_S19_R1_001.fastq.gz":"mp824_R1.fastq.gz",
    "20200103_14_MP7693_S19_R2_001.fastq.gz":"mp824_R2.fastq.gz",
    "20200103_15_MP7693_S20_R1_001.fastq.gz":"mp825_R1.fastq.gz",
    "20200103_15_MP7693_S20_R2_001.fastq.gz":"mp825_R2.fastq.gz",
    "20200103_16_MP7693_S21_R1_001.fastq.gz":"mp826_R1.fastq.gz",
    "20200103_16_MP7693_S21_R2_001.fastq.gz":"mp826_R2.fastq.gz",
    "20200103_17_MP7693_S22_R1_001.fastq.gz":"mp827_R1.fastq.gz",
    "20200103_17_MP7693_S22_R2_001.fastq.gz":"mp827_R2.fastq.gz",
    "20200103_18_MP7693_S23_R1_001.fastq.gz":"mp828_R1.fastq.gz",
    "20200103_18_MP7693_S23_R2_001.fastq.gz":"mp828_R2.fastq.gz",
    "20200103_19_MP7693_S24_R1_001.fastq.gz":"mp829_R1.fastq.gz",
    "20200103_19_MP7693_S24_R2_001.fastq.gz":"mp829_R2.fastq.gz",
    "20200103_20_MP7693_S25_R1_001.fastq.gz":"mp830_R1.fastq.gz",
    "20200103_20_MP7693_S25_R2_001.fastq.gz":"mp830_R2.fastq.gz"
}

In [ ]:
for k,v in rename.items():
    ! mv $bamfolder$k $bamfolder$v

In [ ]:
inputfastq="gs://amlproject/Chip_AML/Chip/mp99.fastq.gz"
! gsutil cp $inputfastq $bamfolder

In [ ]:
a = ! ls $bamfolder

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
df = {
"fastq1": [],
"fastq2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-1],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('_')[0]]
    df['group'].append((row.id.values[0] + '_'+ row.name_replicate.values[0] +"_"+row.protein.values[0]))
    df['replicate'].append(1)
    df['fastq1'].append(val[0])
    #df['fastq2'].append(val[1])
    df['antibody'].append(row.protein.values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq1'].append(a[-1])
#df['fastq2'].append(val[1])
df['antibody'].append("INPUT")
df['control'].append("INPUT")
df = pd.DataFrame(df)

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv')

In [ ]:
#process chips
 ! sudo ./nextflow run nf-core/chipseq --single_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --narrow_peak --input design.csv --genome GRCh38 --skip_preseq --max_cpus 24 -profile docker -w work

In [ ]:
! gsutil -m cp -r results gs://amlproject/Chip_AML/IRF2BP2/histone_results/
! gsutil -m cp $bamfolder'/*' gs://amlproject/Chip_AML/IRF2BP2/histone_results/
! mv results results_hist

In [ ]:
scales = [[1.0,0.6866626897],
[1.0,0.5552893267],
[1.0,0.5290714207],
[0.7295919826,1.0],
[0.5569120906,1.0],
[1.0,0.9361199825],
[1.0,0.6189290029],
[1.0,0.2707451142],
[1.0,0.762887513],
[1.0,0.9422693098]]

In [ ]:
# diffPeak & 
bams = ! ls results_hist/bwa/mergedLibrary/mp*.bam
for i in range(int(len(bams[1:])/2)-1):
    name1 = bams[1+val]
    name2 = bams[9+val]
    chip.fullDiffPeak(name1,name2, bams[0], scaling = scales[val])

In [ ]:
scales = [1.0,
1.0,
1.0,
0.7295919826,
0.5569120906,
1.0,
1.0,
1.0,
1.0,
1.0,
0.6866626897,
0.5552893267,
0.5290714207,
1.0,
1.0,
0.9361199825,
0.6189290029,
0.2707451142,
0.762887513,
0.9422693098,]

In [ ]:
chip.bigWigFrom(bams[1:],genome='GRCh38',scaling=scales)

In [ ]:
! gsutil -m cp -r "diffPeaks" gs://amlproject/Chip_AML/IRF2BP2/ &&\
    gsutil -m cp -r "diffData" gs://amlproject/Chip_AML/IRF2BP2/ &&\
        gsutil -m cp "bigwigs/mp*" gs://amlproject/Chip_AML/IRF2BP2/bigwigs

wc: 0: No such file or directory

wc: 0: No such file or directory



## v3

In [4]:
! gsutil mv gs://transfer-amlproject/*MP7781*  gs://transfer-amlproject/IRF2BP2_v3/

Copying gs://transfer-amlproject/20200203_10_MP7781_S76_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://transfer-amlproject/20200203_10_MP7781_S76_R1_001.fastq.gz...     
Copying gs://transfer-amlproject/20200203_10_MP7781_S76_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://transfer-amlproject/20200203_10_MP7781_S76_R2_001.fastq.gz...     
Copying gs://transfer-amlproject/20200203_11_MP7781_S77_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://transfer-amlproject/20200203_11_MP7781_S77_R1_001.fastq.gz...     
Copying gs://transfer-amlproject/20200203_11_MP7781_S77_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://transfer-amlproject/20200203_11_MP7781_S77_R2_001.fastq.gz...     

==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about 

In [7]:
ls

CRIPSR_guides/  download urls/  genome_size.tsv  RNA/         slamseq_data/
CTF.csv         genes.bed       GEO/             SEgenes.csv  slamseq output/
data_README.md  genesets/       jq1.mv411.csv    slamseq.csv


In [10]:
! mkdir bams && gsutil -m cp gs://transfer-amlproject/IRF2BP2_v3/* bams/

Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_10_MP7781_S76_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_10_MP7781_S76_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_11_MP7781_S77_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_11_MP7781_S77_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_12_MP7781_S78_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_12_MP7781_S78_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_13_MP7781_S79_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_13_MP7781_S79_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_14_MP7781_S80_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_14_MP7781_S80_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_1_MP7781_S67_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v3/20200203_1_

In [13]:
cd bams

/home/jeremie/AMLproject/data/bams


In [15]:
rename = {
"20200203_1_MP7781_S67_R1_001.fastq.gz":"mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.fastq.gz",
"20200203_1_MP7781_S67_R2_001.fastq.gz":"mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R2.fastq.gz",
"20200203_2_MP7781_S68_R1_001.fastq.gz":"mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.fastq.gz",
"20200203_2_MP7781_S68_R2_001.fastq.gz":"mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R2.fastq.gz",
"20200203_3_MP7781_S69_R1_001.fastq.gz":"mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.fastq.gz",
"20200203_3_MP7781_S69_R2_001.fastq.gz":"mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R2.fastq.gz",
"20200203_4_MP7781_S70_R1_001.fastq.gz":"mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.fastq.gz",
"20200203_4_MP7781_S70_R2_001.fastq.gz":"mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R2.fastq.gz",
"20200203_5_MP7781_S71_R2_001.fastq.gz":"mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R2.fastq.gz",
"20200203_5_MP7781_S71_R1_001.fastq.gz":"mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.fastq.gz",
"20200203_6_MP7781_S72_R1_001.fastq.gz":"mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.fastq.gz",
"20200203_6_MP7781_S72_R2_001.fastq.gz":"mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R2.fastq.gz",
"20200203_7_MP7781_S73_R1_001.fastq.gz":"mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.fastq.gz",
"20200203_7_MP7781_S73_R2_001.fastq.gz":"mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R2.fastq.gz",
"20200203_8_MP7781_S74_R1_001.fastq.gz":"mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.fastq.gz",
"20200203_8_MP7781_S74_R2_001.fastq.gz":"mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R2.fastq.gz",
"20200203_9_MP7781_S75_R1_001.fastq.gz":"mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.fastq.gz",
"20200203_9_MP7781_S75_R2_001.fastq.gz":"mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R2.fastq.gz",
"20200203_10_MP7781_S76_R2_001.fastq.gz":"mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R2.fastq.gz",
"20200203_10_MP7781_S76_R1_001.fastq.gz":"mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.fastq.gz",
"20200203_11_MP7781_S77_R1_001.fastq.gz":"mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.fastq.gz",
"20200203_11_MP7781_S77_R2_001.fastq.gz":"mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R2.fastq.gz",
"20200203_12_MP7781_S78_R1_001.fastq.gz":"mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1.fastq.gz",
"20200203_12_MP7781_S78_R2_001.fastq.gz":"mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R2.fastq.gz",
"20200203_13_MP7781_S79_R1_001.fastq.gz":"mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1.fastq.gz",
"20200203_13_MP7781_S79_R2_001.fastq.gz":"mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R2.fastq.gz",
"20200203_14_MP7781_S80_R2_001.fastq.gz":"mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R2.fastq.gz",
"20200203_14_MP7781_S80_R1_001.fastq.gz":"mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R1.fastq.gz",
"20200203_Input_MP7781_S81_R1_001.fastq.gz":"mp845-MV411_IRF2BP2_-INPUT-r1_R1.fastq.gz",
"20200203_Input_MP7781_S81_R2_001.fastq.gz":"mp845-MV411_IRF2BP2_-INPUT-r1_R2.fastq.gz"
}

In [16]:
for k,v in rename.items():
    ! mv $k $v

In [18]:
ls

mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.fastq.gz
mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R2.fastq.gz
mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.fastq.gz
mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R2.fastq.gz
mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.fastq.gz
mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R2.fastq.gz
mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.fastq.gz
mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R2.fastq.gz
mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.fastq.gz
mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R2.fastq.gz
mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.fastq.gz
mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R2.fastq.gz
mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.fastq.gz
mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R2.fastq.gz
mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.fastq.gz
mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R2.fastq.gz
mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.fastq.gz
mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R2.fastq.gz
mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.fastq.gz
mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R2.fastq.gz
mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.fastq.gz
mp841-MV411_IRF2BP

In [19]:
cd ..

/home/jeremie/AMLproject/data


In [ ]:
! gsutil ls gs://amlproject/Chip_AML/IRF2BP2/

In [20]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [52]:
a = ! ls bams

In [54]:
gsheet

,id,cell line,replicate,quality,protein,matching input name,processed,name,previous name,bigwig,...,diffPeaks,individual QC,FLAGTOCOMP,total reads,mapped reads (hg38),unique mapped reads (hg38),multiple mapped reads (hg38),mapped read (droso),unique mapped reads(droso),scaling factor
0,mp100,U937,1,NaN,INPUT,NaN,Y,mp100-U937-INPUT-r1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mp101,NOMO1,1,NaN,INPUT,NaN,Y,mp101-NOMO1-INPUT-r1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mp102,UT7,1,NaN,INPUT,NaN,Y,mp102-UT7-INPUT-r1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mp106,MV411,1,x,MYB,INPUT_MV411,Y,mp106-MV411-MYB-r1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mp109,M6,1,x,CEBPA,INPUT_M6,Y,mp109-M6-CEBPA-r1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,mp841,MV411_IRF2BP_VHL_6h,1,NaN,SMC1,NaN,NaN,mp841-MV411_IRF2BP_VHL_6h-SMC1-r1,20200203_11_MP7781_S77_R1_001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,mp842,MV411_IRF2BP_VHL_6h,3,NaN,MED1,NaN,NaN,mp842-MV411_IRF2BP_VHL_6h-MED1-r3,20200203_12_MP7781_S78_R1_001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,mp843,MV411_IRF2BP_VHL_6h,1,NaN,ZEB2,NaN,NaN,mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1,20200203_13_MP7781_S79_R1_001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,mp844,MV411_IRF2BP_VHL_6h,1,NaN,CEBPA,NaN,NaN,mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1,20200203_14_MP7781_S80_R1_001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df = {
"fastq_1": [],
"fastq_2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-2],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('-')[0]]
    df['group'].append(row['name'].values[0])
    df['replicate'].append(1)
    df['fastq_1'].append('bams/'+val[0])
    df['fastq_2'].append('bams/'+val[1])
    df['antibody'].append(row['protein'].values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq_1'].append('bams/'+a[-2])
df['fastq_2'].append('bams/'+a[-1])
df['antibody'].append("")
df['control'].append("")
df = pd.DataFrame(df)

In [79]:
df

,fastq_1,fastq_2,antibody,group,replicate,control
0,bams/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.fas...,bams/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R2.fas...,CDK8,mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1,1,INPUT
1,bams/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.fas...,bams/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R2.fas...,BRD4,mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1,1,INPUT
2,bams/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.fas...,bams/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R2.fas...,IRF8,mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1,1,INPUT
3,bams/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.fas...,bams/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R2.fas...,SMC1,mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1,1,INPUT
4,bams/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.fas...,bams/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R2.fas...,MED1,mp835-MV411_IRF2BP_DMSO_6h-MED1-r3,1,INPUT
5,bams/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.fas...,bams/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R2.fas...,ZEB2,mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1,1,INPUT
6,bams/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.fa...,bams/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R2.fa...,CEBPA,mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1,1,INPUT
7,bams/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.fast...,bams/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R2.fast...,CDK8,mp838-MV411_IRF2BP_VHL_6h-CDK8-r1,1,INPUT
8,bams/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.fast...,bams/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R2.fast...,BRD4,mp839-MV411_IRF2BP_VHL_6h-BRD4-r1,1,INPUT
9,bams/mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.fast...,bams/mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R2.fast...,IRF8,mp840-MV411_IRF2BP_VHL_6h-IRF8-r1,1,INPUT


In [80]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv',index=False)

In [82]:
#process chips
! sudo ../nextflow run nf-core/chipseq --paired_end --seq_center 'DFCI' --email \
'jkobject@gmail.com' --narrow_peak --input design.csv --genome GRCh38 --skip_preseq \
--max_cpus 24 -profile docker -w work

N E X T F L O W  ~  version 19.10.0
Launching `nf-core/chipseq` [grave_baekeland] - revision: 21be314954 [master]
----------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/chipseq v1.1.0
----------------------------------------------------
Run Name            : grave_baekeland
Data Type           : Paired-End
Design File         : design.csv
Genome              : GRCh38
Fasta File          : s3://ngi-igenomes/igenomes//Homo_sapiens/NCBI/GRCh38/Sequence/WholeGenomeFasta/genome.fa
GTF File            : s3://ngi-igenomes/igenomes//Homo_sapiens/NCBI/GRCh38/Annotation/Genes/genes.gtf
Gene BED File       : s3://ngi-igenomes/igenomes//Homo_sapiens/NCBI/GRCh38/Annotation/Genes/genes.bed
BWA Index           : s3://ngi-igenomes/igenomes//Homo_sapiens


executor >  local (7)
[f1/2ab5d7] process > CheckDesign (design.csv)       [100%] 1 of 1 ✔
[-        ] process > MakeTSSBED                     -
[-        ] process > MakeGenomeFilter               -
[71/1fdcc8] process > FastQC (mp833-MV411_IRF2BP_... [  0%] 0 of 2
[13/a03e87] process > TrimGalore (mp842-MV411_IRF... [  0%] 0 of 2
[-        ] process > BWAMem                         -
[-        ] process > SortBAM                        -
[-        ] process > MergeBAM                       -
[-        ] process > MergeBAMFilter                 -
[-        ] process > MergeBAMRemoveOrphan           -
[-        ] process > Preseq                         -
[-        ] process > CollectMultipleMetrics         -
[-        ] process > BigWig                         -
[-        ] process > PlotProfile                    -
[-        ] process > PhantomPeakQualTools           -
[-        ] process > PlotFingerprint                -
[-        ] process > MACSCallPeak                   -
[-  


executor >  local (12)
[f1/2ab5d7] process > CheckDesign (design.csv)       [100%] 1 of 1 ✔
[-        ] process > MakeTSSBED                     -
[-        ] process > MakeGenomeFilter               -
[37/e12263] process > FastQC (mp839-MV411_IRF2BP_... [ 60%] 3 of 5
[57/f88790] process > TrimGalore (mp838-MV411_IRF... [  0%] 0 of 4
[-        ] process > BWAMem                         -
[-        ] process > SortBAM                        -
[-        ] process > MergeBAM                       -
[-        ] process > MergeBAMFilter                 -
[-        ] process > MergeBAMRemoveOrphan           -
[-        ] process > Preseq                         -
[-        ] process > CollectMultipleMetrics         -
[-        ] process > BigWig                         -
[-        ] process > PlotProfile                    -
[-        ] process > PhantomPeakQualTools           -
[-        ] process > PlotFingerprint                -
[-        ] process > MACSCallPeak                   -
[- 


executor >  local (18)
[f1/2ab5d7] process > CheckDesign (design.csv)       [100%] 1 of 1 ✔
[-        ] process > MakeTSSBED                     -
[-        ] process > MakeGenomeFilter               -
[90/c3be73] process > FastQC (mp843-MV411_IRF2BP_... [ 80%] 8 of 10
[87/e0f172] process > TrimGalore (mp843-MV411_IRF... [ 20%] 1 of 5
[-        ] process > BWAMem                         -
[-        ] process > SortBAM                        -
[-        ] process > MergeBAM                       -
[-        ] process > MergeBAMFilter                 -
[-        ] process > MergeBAMRemoveOrphan           -
[-        ] process > Preseq                         -
[-        ] process > CollectMultipleMetrics         -
[-        ] process > BigWig                         -
[-        ] process > PlotProfile                    -
[-        ] process > PhantomPeakQualTools           -
[-        ] process > PlotFingerprint                -
[-        ] process > MACSCallPeak                   -
[-


executor >  local (23)
[f1/2ab5d7] process > CheckDesign (design.csv)       [100%] 1 of 1 ✔
[-        ] process > MakeTSSBED                     -
[-        ] process > MakeGenomeFilter               -
[9a/6fa4b9] process > FastQC (mp844-MV411_IRF2BP_... [ 85%] 11 of 13
[6c/a36b78] process > TrimGalore (mp833-MV411_IRF... [ 43%] 3 of 7
[-        ] process > BWAMem                         -
[-        ] process > SortBAM                        -
[-        ] process > MergeBAM                       -
[-        ] process > MergeBAMFilter                 -
[-        ] process > MergeBAMRemoveOrphan           -
[-        ] process > Preseq                         -
[-        ] process > CollectMultipleMetrics         -
[-        ] process > BigWig                         -
[-        ] process > PlotProfile                    -
[-        ] process > PhantomPeakQualTools           -
[-        ] process > PlotFingerprint                -
[-        ] process > MACSCallPeak                   -
[

...


executor >  local (214)
[f1/2ab5d7] process > CheckDesign (design.csv)       [100%] 1 of 1 ✔
[8e/f95391] process > MakeTSSBED (genes.bed)         [100%] 1 of 1 ✔
[24/b78c15] process > MakeGenomeFilter (genome.fa)   [100%] 1 of 1 ✔
[b8/7460ba] process > FastQC (INPUT_R1_T1)           [100%] 15 of 15 ✔
[ad/fd3bed] process > TrimGalore (INPUT_R1_T1)       [100%] 15 of 15 ✔
[f8/bd9528] process > BWAMem (INPUT_R1_T1)           [100%] 15 of 15 ✔
[61/9f64ef] process > SortBAM (INPUT_R1_T1)          [100%] 15 of 15 ✔
[b9/a97f1d] process > MergeBAM (mp833-MV411_IRF2B... [100%] 15 of 15 ✔
[5d/ca8e83] process > MergeBAMFilter (mp833-MV411... [100%] 15 of 15 ✔
[c0/c55284] process > MergeBAMRemoveOrphan (mp833... [100%] 15 of 15 ✔
[-        ] process > Preseq                         -
[2f/c077a2] process > CollectMultipleMetrics (mp8... [100%] 15 of 15 ✔
[cc/e4f33b] process > BigWig (mp833-MV411_IRF2BP_... [100%] 15 of 15 ✔
[b1/145924] process > PlotProfile (mp836-MV411_IR... [ 90%] 9 of 10
[60/b6

[cc/e4f33b] process > BigWig (mp833-MV411_IRF2BP_... [100%] 15 of 15 ✔
[8f/0998f3] process > PlotProfile (mp840-MV411_IR... [ 93%] 13 of 14
[60/b6a6d1] process > PhantomPeakQualTools (mp833... [100%] 15 of 15 ✔
[b4/8570e9] process > PlotFingerprint (mp833-MV41... [100%] 14 of 14 ✔
[e2/22b634] process > MACSCallPeak (mp833-MV411_I... [100%] 14 of 14 ✔
[46/188907] process > AnnotatePeaks (mp840-MV411_... [100%] 13 of 13
[-        ] process > PeakQC                         -
[5c/cb5c92] process > ConsensusPeakSet (CEBPA)       [100%] 7 of 7 ✔
[-        ] process > ConsensusPeakSetAnnotate       -
[-        ] process > ConsensusPeakSetDESeq          -
[bd/6dfdda] process > IGV                            [100%] 1 of 1 ✔
[3f/6b0aa2] process > get_software_versions          [100%] 1 of 1 ✔
[-        ] process > MultiQC                        -
[e3/fe1218] process > output_documentation           [100%] 1 of 1 ✔

executor >  local (219)
[f1/2ab5d7] process > CheckDesign (design.csv)       [100


executor >  local (224)
[f1/2ab5d7] process > CheckDesign (design.csv)       [100%] 1 of 1 ✔
[8e/f95391] process > MakeTSSBED (genes.bed)         [100%] 1 of 1 ✔
[24/b78c15] process > MakeGenomeFilter (genome.fa)   [100%] 1 of 1 ✔
[b8/7460ba] process > FastQC (INPUT_R1_T1)           [100%] 15 of 15 ✔
[ad/fd3bed] process > TrimGalore (INPUT_R1_T1)       [100%] 15 of 15 ✔
[f8/bd9528] process > BWAMem (INPUT_R1_T1)           [100%] 15 of 15 ✔
[61/9f64ef] process > SortBAM (INPUT_R1_T1)          [100%] 15 of 15 ✔
[b9/a97f1d] process > MergeBAM (mp833-MV411_IRF2B... [100%] 15 of 15 ✔
[5d/ca8e83] process > MergeBAMFilter (mp833-MV411... [100%] 15 of 15 ✔
[c0/c55284] process > MergeBAMRemoveOrphan (mp833... [100%] 15 of 15 ✔
[-        ] process > Preseq                         -
[2f/c077a2] process > CollectMultipleMetrics (mp8... [100%] 15 of 15 ✔
[cc/e4f33b] process > BigWig (mp833-MV411_IRF2BP_... [100%] 15 of 15 ✔
[7c/94dbc4] process > PlotProfile (mp833-MV411_IR... [100%] 15 of 15 ✔
[60


executor >  local (228)
[f1/2ab5d7] process > CheckDesign (design.csv)       [100%] 1 of 1 ✔
[8e/f95391] process > MakeTSSBED (genes.bed)         [100%] 1 of 1 ✔
[24/b78c15] process > MakeGenomeFilter (genome.fa)   [100%] 1 of 1 ✔
[b8/7460ba] process > FastQC (INPUT_R1_T1)           [100%] 15 of 15 ✔
[ad/fd3bed] process > TrimGalore (INPUT_R1_T1)       [100%] 15 of 15 ✔
[f8/bd9528] process > BWAMem (INPUT_R1_T1)           [100%] 15 of 15 ✔
[61/9f64ef] process > SortBAM (INPUT_R1_T1)          [100%] 15 of 15 ✔
[b9/a97f1d] process > MergeBAM (mp833-MV411_IRF2B... [100%] 15 of 15 ✔
[5d/ca8e83] process > MergeBAMFilter (mp833-MV411... [100%] 15 of 15 ✔
[c0/c55284] process > MergeBAMRemoveOrphan (mp833... [100%] 15 of 15 ✔
[-        ] process > Preseq                         -
[2f/c077a2] process > CollectMultipleMetrics (mp8... [100%] 15 of 15 ✔
[cc/e4f33b] process > BigWig (mp833-MV411_IRF2BP_... [100%] 15 of 15 ✔
[7c/94dbc4] process > PlotProfile (mp833-MV411_IR... [100%] 15 of 15 ✔
[60


executor >  local (229)
[f1/2ab5d7] process > CheckDesign (design.csv)       [100%] 1 of 1 ✔
[8e/f95391] process > MakeTSSBED (genes.bed)         [100%] 1 of 1 ✔
[24/b78c15] process > MakeGenomeFilter (genome.fa)   [100%] 1 of 1 ✔
[b8/7460ba] process > FastQC (INPUT_R1_T1)           [100%] 15 of 15 ✔
[ad/fd3bed] process > TrimGalore (INPUT_R1_T1)       [100%] 15 of 15 ✔
[f8/bd9528] process > BWAMem (INPUT_R1_T1)           [100%] 15 of 15 ✔
[61/9f64ef] process > SortBAM (INPUT_R1_T1)          [100%] 15 of 15 ✔
[b9/a97f1d] process > MergeBAM (mp833-MV411_IRF2B... [100%] 15 of 15 ✔
[5d/ca8e83] process > MergeBAMFilter (mp833-MV411... [100%] 15 of 15 ✔
[c0/c55284] process > MergeBAMRemoveOrphan (mp833... [100%] 15 of 15 ✔
[-        ] process > Preseq                         -
[2f/c077a2] process > CollectMultipleMetrics (mp8... [100%] 15 of 15 ✔
[cc/e4f33b] process > BigWig (mp833-MV411_IRF2BP_... [100%] 15 of 15 ✔
[7c/94dbc4] process > PlotProfile (mp833-MV411_IR... [100%] 15 of 15 ✔
[60

In [89]:
ls

bams/              diffBinding_rep1/  human_bams_rep2/      ref/
bigwig/            diffBinding_rep2/  MV4narrow/            results/
binding_profile/   fastq1/            narrowPeak/           results_hist/
BroadPeaks/        fastq2/            recalib_bigwig_hist/  slam/
design.csv         fastq_hist/        recalib_bigwig_rep1/  temp/
diffBinding_hist/  human_bams_rep1/   recalib_bigwig_rep2/  work/


In [91]:
!gsutil -m cp -r results/* gs://amlproject/Chip/IRF2BP2_degraded_rep3/ && sudo rm -r work && mv results results3

In [100]:
ls ../TrimGalore-0.6.5/trim_galore

bams/              diffBinding_rep1/  human_bams_rep2/      ref/
bigwig/            diffBinding_rep2/  MV4narrow/            results3/
binding_profile/   fastq1/            narrowPeak/           results_hist/
BroadPeaks/        fastq2/            recalib_bigwig_hist/  slam/
design.csv         fastq_hist/        recalib_bigwig_rep1/  temp/
diffBinding_hist/  human_bams_rep1/   recalib_bigwig_rep2/


In [106]:
# get scaling values
mappedreads, umappedreads_norm = chip.getSpikeInControlScales(refgenome="ref/reference_droso.fna",
                                                                      fastQfolder='res/',
                                                                      pairedEnd=True, cores=8,
                                                                      tofilter=False,
                                                                      totrim=False,
                                                                      tomap=False,
                                                                      pathtotrim_galore="../TrimGalore-0.6.5/trim_galore")
mappedreads, umappedreads_norm

if paired_end, need to be name_*1, name_*2
using all files from folder
['mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.fastq.gz_trimming_report.txt', 'mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_val_1.fq.gz', 'mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_val_1.mapped.sam', 'mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_val_1.sorted.bam', 'mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_val_1.sorted.bam.bai', 'mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_val_1.sorted.bam.flagstat', 'mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_val_1.sorted.bam.idxstat', 'mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R2.fastq.gz_trimming_report.txt', 'mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R2_val_2.fq.gz', 'mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.fastq.gz_trimming_report.txt', 'mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_val_1.fq.gz', 'mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_val_1.mapped.sam', 'mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_val_1.sorted.bam', 'mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_val_1.sorted.bam.bai', 'mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_val_1.sorted.bam.flagstat', 'mp

({'mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_val_1': 0.007380946616181464,
  'mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_val_1': 0.006589952940785207,
  'mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1_val_1': 0.0016497391139281609,
  'mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1_val_1': 0.009483311477606655,
  'mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1_val_1': 0.0025657294168679066,
  'mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1_val_1': 0.0033727831801694304,
  'mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1_val_1': 0.0136827386192967,
  'mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_val_1': 0.006265038146699591,
  'mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1_val_1': 0.005815924450216832,
  'mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1_val_1': 0.002364272107853285,
  'mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1_val_1': 0.018330928063943163,
  'mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1_val_1': 0.002933812653519909,
  'mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1_val_1': 0.002878568429738379,
  'mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R1_val_1': 0.016487356780909113,
  'mp845-M

computing scales from the excel sheet

In [108]:
scales = [[1.0,0.8488122828],
[1.0,0.8825441551],
[0.6977788675,1.0],
[0.5173394083,1.0],
[0.8745375796,1.0],
[1.0,0.8534697536],
[0.8298927961,1.0]]

In [74]:
bams = ! ls ../../data/results3/bwa/mergedLibrary/*.bam
bams

['../../data/results3/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/

In [106]:
bams[7]

'../../data/results3/bwa/mergedLibrary/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam'

In [118]:
# diffPeak & 
for i in range(int(len(bams[1:])/2)-1):
    name1 = bams[1+i]
    name2 = bams[8+i]
    chip.fullDiffPeak(name1,name2, bams[0], scaling = scales[i])

doing diff from ../../data/results3/bwa/mergedLibrary/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.sorted.bam and ../../data/results3/bwa/mergedLibrary/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/results3/bwa/mergedLibrary/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.sorted.bam and ../../data/results3/bwa/mergedLibrary/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/results3/bwa/mergedLibrary/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.sorted.bam and ../../data/results3/bwa/mergedLibrary/mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/results3/bwa/mergedLibrary/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.sorted.bam and ../../data/results3/bwa/mergedLibrary/mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/results3/bwa/mergedLibrary/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.sorted.bam and ../../data/results3/bwa/mergedLibrary/mp842-MV411

In [125]:
mkdir ../../data/IRF2BP2_degraded_3/

In [126]:
! mv diffPeaks ../../data/IRF2BP2_degraded_3
! mv diffData ../../data/IRF2BP2_degraded_3

In [127]:
os.popen('for i in $(ls ../../data/IRF2BP2_degraded_3/diffPeaks); \
            do echo $(wc -l "../../data/IRF2BP2_degraded_3/diffPeaks/"$i); \
            done').read().split('\n')

['8014 ../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_common.bed',
 '305 ../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_cond1.bed',
 '2476 ../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_cond2.bed',
 '23311 ../../data/IRF2BP2_degraded_3/diffPeaks/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_vs_mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1_c3.0_common.bed',
 '99 ../../data/IRF2BP2_degraded_3/diffPeaks/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_vs_mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1_c3.0_cond1.bed',
 '3424 ../../data/IRF2BP2_degraded_3/diffPeaks/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_vs_mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1_c3.0_cond2.bed',
 '39726 ../../data/IRF2BP2_degraded_3/diffPeaks/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1_vs_mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1_c3.0_common.bed',
 '20

In [128]:
scales = [1.0,
1.0,
0.6977788675,
0.5173394083,
0.8745375796,
1.0,
0.8298927961,
0.8488122828,
0.8825441551,
1.0,
1.0,
1.0,
0.8534697536,
1.0]

In [130]:
bams

['../../data/results3/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.mLb.clN.sorted.bam',
 '../../data/results3/bwa/mergedLibrary/

In [134]:
chip.bigWigFrom(bams[1:], 
                genome='GRCh38',scaling=scales,
               numthreads=8)

In [138]:
!mv bigwig ../../data/recalib_bigwig_3 

In [140]:
ls

ChIP_analysis.ipynb                     processing ChipSeq.ipynb
ChIP_analysis_step_2.ipynb              Processing_IRF2BP2_degraded_MV411.ipynb
creating_intervals_CRISPRtargets.ipynb  RNA-AMLproject.ipynb
DeepBind_Analysis.ipynb                 RNA_RNP_analysis.ipynb
IRF2BP2 ChIPs Analysis.ipynb            slamRNA_analysis.ipynb
JQ1_RNA_analysis-v1.ipynb               slamseq IRF2BP2 degraded.ipynb
JQ1_RNA_analysis-v2.ipynb               slamseq_MYCpaper.ipynb
plot with K562.ipynb


In [150]:
! gsutil -m cp -r "../../data/recalib_bigwig_3/*" gs://amlproject/Chip/IRF2BP2_degraded_rep3/recalib_bigwig/ &&\
gsutil -m cp -r "../../data/IRF2BP2_degraded_3/diffData/*" gs://amlproject/Chip/IRF2BP2_degraded_rep3/diffData/ &&\
gsutil -m cp -r "../../data/IRF2BP2_degraded_3/diffPeaks/*" gs://amlproject/Chip/IRF2BP2_degraded_rep3/diffPeaks/

Copying file://../../data/IRF2BP2_degraded_3/diffPeaks/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1_vs_mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1_c3.0_cond1.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_3/diffPeaks/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1_vs_mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1_c3.0_common.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_3/diffPeaks/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1_vs_mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1_c3.0_cond2.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_common.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_3/diffPeaks/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1_vs_mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1_c3.0_cond2.bed [Content-Type=application/octet-stream]...
Copying file://../../data/IRF2BP2_degraded_

## RNA

In [ ]:
a = !gsutil ls gs://amlproject/slamseq/IRF2BP2/

In [ ]:
rename1 = {
'gs://amlproject/slamseq/IRF2BP2/20191212_10_MP7623_S30_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp810_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_10_MP7623_S30_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp810_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_11_MP7623_S31_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp811_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_11_MP7623_S31_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp811_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_12_MP7623_S32_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp812_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_12_MP7623_S32_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp812_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_13_MP7623_S33_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp813_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_13_MP7623_S33_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp813_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_14_MP7623_S34_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp814_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_14_MP7623_S34_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp814_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_15_MP7623_S35_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp815_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_15_MP7623_S35_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp815_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_16_MP7623_S36_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp816_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_16_MP7623_S36_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp816_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_1_MP7623_S21_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp801_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_1_MP7623_S21_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp801_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_2_MP7623_S22_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp802_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_2_MP7623_S22_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp802_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_3_MP7623_S23_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp803_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_3_MP7623_S23_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp803_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_4_MP7623_S24_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp804_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_4_MP7623_S24_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp804_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_5_MP7623_S25_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp805_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_5_MP7623_S25_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp805_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_6_MP7623_S26_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp806_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_6_MP7623_S26_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp806_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_7_MP7623_S27_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp807_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_7_MP7623_S27_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp807_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_8_MP7623_S28_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp808_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_8_MP7623_S28_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp808_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_9_MP7623_S29_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp809_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_9_MP7623_S29_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp809_2.fastq.gz'}

In [ ]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [ ]:
a

In [ ]:
terra.list_blobs_with_prefix("amlproject","slamseq/IRF2BP2/")

In [ ]:
terra.uploadFromFolder("amlproject", "slamseq/IRF2BP2/", workspace="broad-firecloud-ccle/hg38_RNAseq", 
                       fformat="fastq12", source="maximP",samplesetname="slamseq_IRF2BP2")

In [ ]:
samplesetname = 'slamseq_IRF2BP2'
refworkspace="broad-firecloud-ccle/hg38_RNAseq"
refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
submission_id = refwm.create_submission("slamdunk", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
submission_id1 = refwm.create_submission("rsem_v1-0_BETA_cfg", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id1)

In [ ]:
! pip3 install git+https://github.com/jkobject/slamdunk.git --upgrade

In [65]:
folder_bams = "../../data/slam/data/"
bams = ! ls $folder_bams*.fastq.gz
bams

['../../data/slam/data/mr74-MV411_IRF2BP2-DMSO_2h-r1_1.fastq.gz',
 '../../data/slam/data/mr74-MV411_IRF2BP2-DMSO_2h-r1_2.fastq.gz',
 '../../data/slam/data/mr75-MV411_IRF2BP2-DMSO_2h-r2_1.fastq.gz',
 '../../data/slam/data/mr75-MV411_IRF2BP2-DMSO_2h-r2_2.fastq.gz',
 '../../data/slam/data/mr76-MV411_IRF2BP2-DMSO_2h-r3_1.fastq.gz',
 '../../data/slam/data/mr76-MV411_IRF2BP2-DMSO_2h-r3_2.fastq.gz',
 '../../data/slam/data/mr77-MV411_IRF2BP2-DMSO_2h-r4_1.fastq.gz',
 '../../data/slam/data/mr77-MV411_IRF2BP2-DMSO_2h-r4_2.fastq.gz',
 '../../data/slam/data/mr78-MV411_IRF2BP2-DMSO_6h-r1_1.fastq.gz',
 '../../data/slam/data/mr78-MV411_IRF2BP2-DMSO_6h-r1_2.fastq.gz',
 '../../data/slam/data/mr79-MV411_IRF2BP2-DMSO_6h-r2_1.fastq.gz',
 '../../data/slam/data/mr79-MV411_IRF2BP2-DMSO_6h-r2_2.fastq.gz',
 '../../data/slam/data/mr80-MV411_IRF2BP2-DMSO_6h-r3_1.fastq.gz',
 '../../data/slam/data/mr80-MV411_IRF2BP2-DMSO_6h-r3_2.fastq.gz',
 '../../data/slam/data/mr81-MV411_IRF2BP2-DMSO_6h-r4_1.fastq.gz',
 '../../da

In [ ]:
for val in h.grouped(bams[4:],2):
    print(val)
    res = os.system('../../TrimGalore-0.6.5/trim_galore --paired --cores 12 --retain_unpaired -stringency 3 --illumina '+
    val[0] + ' '+val[1]+" -o "+folder_bams)
    if res!=0:
        raise ValueError()
    res = os.system('slamdunk all -r ../../data/slam/ref_genome.fa -b ../../data/slam/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed \
    -o ../../data/slam/resIRF2BP2/ -t 12 -c 2  '+'.'.join(val[0].split('.')[:-2])+'_val_1.fq.gz \
    '+'.'.join(val[1].split('.')[:-2])+"_val_2.fq.gz")
    if res!=0:
        raise ValueError()

('../../data/slam/data/mr76-MV411_IRF2BP2-DMSO_2h-r3_1.fastq.gz', '../../data/slam/data/mr76-MV411_IRF2BP2-DMSO_2h-r3_2.fastq.gz')
('../../data/slam/data/mr77-MV411_IRF2BP2-DMSO_2h-r4_1.fastq.gz', '../../data/slam/data/mr77-MV411_IRF2BP2-DMSO_2h-r4_2.fastq.gz')
('../../data/slam/data/mr78-MV411_IRF2BP2-DMSO_6h-r1_1.fastq.gz', '../../data/slam/data/mr78-MV411_IRF2BP2-DMSO_6h-r1_2.fastq.gz')
('../../data/slam/data/mr79-MV411_IRF2BP2-DMSO_6h-r2_1.fastq.gz', '../../data/slam/data/mr79-MV411_IRF2BP2-DMSO_6h-r2_2.fastq.gz')


In [ ]:
for val in h.grouped(bams[:4],2):
    print(val)
    res = os.system('slamdunk all -r ../../data/slam/ref_genome.fa -b ../../data/slam/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed \
    -o ../../data/slam/resIRF2BP2/ -t 12 -c 2  '+'.'.join(val[0].split('.')[:-2])+'_val_1.fq.gz \
    '+'.'.join(val[1].split('.')[:-2])+"_val_2.fq.gz")
    if res!=0:
        raise ValueError()